This building model is a simplified faster version of the full two-zone building model. It assume the building is always at thermal equilibrium unless the set-point temperature is changed.

# Building dynamics
Buildings can be simulated using a reduced-order model using a discrete transfer function based on the thermal properties of the building.  The transfer function can be derived for a two-variable model using the indoor air temperature $T_A$ and building mass temperature $T_M$ as they respond to changes in outdoor air temperature $T_O$ and changes in heat gains to the air $Q_A$ and mass $Q_M$, i.e.,
$$
    C_A \dot T_A = U_A ( T_O - T_A ) + U_I ( T_M - T_A ) + Q_A \qquad (1)
$$
$$
    C_M \dot T_M = U_I (T_A - T_M ) + U_M ( T_O - T_M ) + Q_M \qquad (2)
$$
where
* $C_A$ is the heat capacity of the air volume;
* $C_M$ is the heat capacity of the building mass;
* $U_A$ is the conductance between the indoor to outdoor air;
* $U_I$ is the conductance between the indoor air and the building mass,; and
* $U_M$ is the conductance between the building mass and the outdoor air.

The heat gains to the air come from the following sources:
* $Q_E E$, the internal enduse electric loads, where $Q_E$ is installed capacity of electric end-uses and $E$ is the fraction of electric end-uses active;
* $Q_G G$, the internal enduse gas loads, where $Q_G$ is the heat retained in the building for installed capacity of gas enduses and $G$ is the fraction of gas end-uses active;
* $Q_O H$, the occupancy heat gains where $Q_O$ is the heat gain for a single person, and $H$ is the number of people present;
* $Q_V H$, the ventilation heat gains, where $Q_V$ is the heat gain for fresh air per person; and
* $M Q_H$, the HVAC heat gains, where $M$ is the system mode, and $Q_H$ is the nameplate system capacity.
Thus
$$
    Q_A = E Q_E + G Q_G + H (Q_O+Q_V) + M Q_H
$$

The heat gains to the mass from the solar gains to building envelope and mass through windows, such that $Q_M = S_A Q_S$, where $S_A$ is the solar exposure area of the building mass, and $Q_S$ is the total insolation.

The HVAC system mode is included as a state variable such that the mode $M$ represents the state of the HVAC system given the indoor air temperature setpoint $T_S$ deviation from $T_A$ with a proportional gain $K$, where $M>0$ for heating, and $M<0$ for cooling. 

# System dynamics
The state space model is thus
$$
    \left[ \begin{matrix} \dot T_A \\ \dot T_M \\ \dot M \end{matrix} \right]
    =
    \left[ \begin{matrix} 
        -{U_A+U_I \over C_A} & {U_I \over C_A} & {Q_H \over C_A}
    \\
        {U_I \over C_M} & -{U_M+U_I \over C_M} & 0
    \\
        K & 0 & 0
    \end{matrix} \right]
    \left[ \begin{matrix} T_A \\ T_M \\ M \end{matrix} \right]
    +
    \left[ \begin{matrix} 
        { U_A \over C_A } & {Q_E \over C_A} & {Q_G \over C_A} & {Q_O+Q_V \over C_A} & 0 & 0
    \\
        { U_M \over C_M } & 0 & 0 & 0 & {S_A \over C_M} & 0
    \\
        0 & 0 & 0 & 0 & 0 & -K
    \end{matrix} \right]
    \left[ \begin{matrix} T_O \\ E \\ G \\ H \\ Q_S \\ T_S \end{matrix} \right]
    \qquad(3)
$$

# System Sizing

If the system size is not given, then $Q_H$ must be calculated for the design conditions.  The heating system capacity is found by solving Eq. (3) at steady-state for $Q_H$ given the $T_O=T_H$, $T_A=T_S$, $E=G=H=Q_S=0$, and $M=1/D_F$, the system over-design factor. This gives us the equation
$$
    \left[ \begin{matrix} 0 \\ 0 \end{matrix} \right]
    =
    \left[ \begin{matrix} 
        {U_I \over C_A} & {1 \over D_F C_A} \\
        -{U_M+U_I \over C_M} & 0
    \end{matrix} \right]
    \left[ \begin{matrix} T_M \\ Q_H \end{matrix} \right]
    +
    \left[ \begin{matrix} 
        {U_A \over C_A} & -{U_A+U_I \over C_A} \\
        {U_M \over C_M} & {U_I \over C_M}
    \end{matrix} \right]
    \left[ \begin{matrix} T_H \\ T_S \end{matrix} \right],
$$
which we solve for $Q_H$.
The same method is used for cooling but with all internal gains on and maximum solar gain.
$$
    \left[ \begin{matrix} 0 \\ 0 \end{matrix} \right]
    =
    \left[ \begin{matrix} 
        {U_I \over C_A} & {1 \over D_F C_A} \\
        -{U_M+U_I \over C_M} & 0
    \end{matrix} \right]
    \left[ \begin{matrix} T_M \\ Q_H \end{matrix} \right]
    +
    \left[ \begin{matrix} 
        {U_A \over C_A} & -{U_A+U_I \over C_A} & 0 \\
        {U_M \over C_M} & {U_I \over C_M} & { QE+QG+QO+QV+1300S_A \over CA }
    \end{matrix} \right]
    \left[ \begin{matrix} T_H \\ T_S \\ 1 \end{matrix} \right].
$$
The required system capacity is the greater of these two values.

# State space model
The canonical state-space model is for this system is
$$
    \dot x = A~x + B~u
$$
$$
    \dot y = C~x + D~u
$$
where
* $ x = \left[ \begin{matrix} T_A \\ T_M \end{matrix} \right] $,
* $ A = \left[ \begin{matrix}
        {U_A+U_I \over C_A} & -{U_I \over C_A}
    \\
        -{U_A \over C_M} & {U_M+U_I \over C_M}
    \end{matrix} \right] $,
* $ B = \left[ \begin{matrix}
        -{U_A \over C_A} & {1 \over C_A} & 0
    \\
        -{U_M \over C_A} & 0 & {1 \over C_M}
    \end{matrix} \right] $,
* $ u = \left[ \begin{matrix} T_O \\ Q_A \\ Q_M \end{matrix} \right] $,
* $ y = \left[ \begin{matrix} P \\ Q \end{matrix} \right] $,
* $ C = \left[ \begin{matrix} \end{matrix} \right] $,
* $ D = \left[ \begin{matrix}
        -{U_A \over C_A} & {1 \over C_A} & 0
    \\
        -{U_M \over C_A} & 0 & {1 \over C_M}
    \end{matrix} \right] $,

# Equilibrium Conditions
The equilibrium condition $\bar x$ is computed by solving for $\dot x = 0$, i.e.,
$$
    \bar x = -A^{-1} B u,
$$
the result of include the duty cycle $\bar M$ required to maintain the indoor air temperature setpoint.